In [0]:
#!pip3 install jieba

In [0]:

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip3 install sacrebleu

  Running setup.py bdist_wheel for sacrebleu ... - done
  Stored in directory: /root/.cache/pip/wheels/ea/0a/7d/ddcbdcd15a04b72de1b3f78e7e754aab415aff81c423376385
Successfully built sacrebleu


In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x564c0000 @  0x7f536c3542a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
0.4.1
True


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import operator
from torch.utils.data import Dataset
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sacrebleu import corpus_bleu, TOKENIZERS, DEFAULT_TOKENIZER
#from masked_cross_entropy import *
from torch.optim.lr_scheduler import ExponentialLR
from torch.autograd import Variable
import random
from torch.nn import functional

In [0]:
#device = torch.device('cpu')

In [0]:
def filter_pairs(pairs):
    filtered_pairs = []
    for pair in pairs:
        if len(pair[0].split()) >= MIN_LENGTH and len(pair[0].split()) <= MAX_LENGTH \
            and len(pair[1].split()) >= MIN_LENGTH and len(pair[1].split()) <= MAX_LENGTH:
            filtered_pairs.append(pair)
    return filtered_pairs

In [0]:
def sequence_mask(sequence_length, max_len=None):
  
    """
    Code paraphrased from 
    https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/masked_cross_entropy.py
    """
    
    if max_len is None:
        max_len = sequence_length.data.max()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len).contiguous()
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return seq_range_expand < seq_length_expand



def masked_cross_entropy(logits, target, length):
    """
    Code paraphrased from 
    https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/masked_cross_entropy.py
    """
    
    
    """
    Args:
        logits: A Variable containing a FloatTensor of size
            (batch, max_len, num_classes) which contains the
            unnormalized probability for each class.
        target: A Variable containing a LongTensor of size
            (batch, max_len) which contains the index of the true
            class for each corresponding step.
        length: A Variable containing a LongTensor of size (batch,)
            which contains the length of each data in a batch.

    Returns:
        loss: An average loss value masked by the length.
    """
  
    length = torch.LongTensor(length).to(device)

    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = functional.log_softmax(logits_flat, dim=1)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask.float()
    loss = losses.sum() / length.float().sum()
    return loss

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_token = 0
SOS_token = 1
EOS_token = 2
hidden_size = 512
dropout_p = 0.1
teacher_forcing_ratio = 1
BATCH_SIZE = 64
MIN_LENGTH = 3
MAX_LENGTH = 30
source_vocab_size = 4300
target_vocab_size = 39000
n_layers = 4
lr_rate_en = 0.0001
lr_rate_de = 0.0005
lr_decay = True
gamma_encoder = 0.9
gamma_decoder = 0.9
n_epochs = 20
plot_every = 100
print_every = 100
evaluate_every = 100
attn_model = 'dot'
Attention = True
search_method = 'greedy'
beam_size = 10
n_best = 5
dynamic_sentence_length = False

## Loading Data

In [0]:
class Lang:
 
    '''
    Some codes are paraphrased from
    https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
    '''
    
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
def unicodeToAscii(s):
    """ Turn a Unicode string to plain ASCII, 
    thanks to http://stackoverflow.com/a/518232/2809427"""

    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def normalizeEnString(s):
    s = unicodeToAscii(s.strip())
    s = s.replace("&apos;", "&apos").replace("&quot","")
   
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?0-9]+", r" ", s)
    s = s.replace("apos", "'")
    return s


def normalizeChString(s):
    s = "".join(s.split())
    s = ' '.join(str(x) for x in s)
    return s.strip() 


normalizeEnString("It &apos;s very pretty , and it has rapidly started to overgrow the \
                  once very rich biodiversity of the northwestern Mediterranean .")
normalizeChString('感 应 电 机 可      以 加 热 东    西     ，     尤 其 擅 长 加 热 钢 铁')               

'感 应 电 机 可 以 加 热 东 西 ， 尤 其 擅 长 加 热 钢 铁'

In [0]:
def readLangs(lang1, lang2, data='train'):
  
    '''
    Some codes are paraphrased from
    https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
    '''
    
    
    #data: train/dev/test
    print("Reading lines...")
    # Read the file and split into lines
    zh_lines = open('/content/drive/My Drive/Colab Notebooks/Neural-Machine-Translation/iwslt-zh-en-sn/{}.tok.zh'.format(data)).read().split('\n')
    en_lines = open('/content/drive/My Drive/Colab Notebooks/Neural-Machine-Translation/iwslt-zh-en-sn/{}.tok.en'.format(data)).read().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeChString(element[0]), normalizeEnString(element[1])] for element in zip(zh_lines, en_lines)]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

def build_topwordVocab(lang, vocab_size):
    print("Build vocabulary by top {} frequent word...".format(vocab_size))
    sorted_word2Count = sorted(lang.word2count.items(),
        key=operator.itemgetter(1),
        reverse=True)
    sorted_words = [x[0] for x in sorted_word2Count[:vocab_size]]
    
    lang.word2index = {}

    for ind, word in enumerate(sorted_words):
            lang.word2index[word] = ind + 4
    lang.index2word = {}
    lang.index2word[0] = "<PAD>"
    lang.index2word[1] = "<SOS>"
    lang.index2word[2] = "<EOS>"
    lang.index2word[3] = "<UNK>"

    for ind, word in enumerate(sorted_words):
        lang.index2word[ind + 4] = word
    
    lang.n_words = len(lang.index2word)
    
    print(lang.name, lang.n_words)
    return lang

input_lang, output_lang, pairs = prepareData('ch', 'eng')

input_lang = build_topwordVocab(input_lang,vocab_size=source_vocab_size)
output_lang = build_topwordVocab(output_lang, vocab_size=target_vocab_size)
print(random.choice(pairs))

Reading lines...
Read 213378 sentence pairs
Trimmed to 134273 sentence pairs
Counting words...
Counted words:
ch 4476
eng 40276
Build vocabulary by top 4300 frequent word...
ch 4304
Build vocabulary by top 39000 frequent word...
eng 39004
['现 实 世 界 中 数 学 并 不 是 数 学 家 的 专 用 品 。', "See in the real world math isn 't necessarily done by mathematicians ."]


In [0]:
_, _, val_pairs = readLangs('ch', 'eng', 'dev')
val_pairs = filter_pairs(val_pairs[:-1])
_, _, test_pairs = readLangs('ch', 'eng', 'test')
test_pairs = filter_pairs(test_pairs)

Reading lines...
Reading lines...


In [0]:
print(random.choice(val_pairs))

['这 是 南 洛 杉 矶 （ 笑 ）', 'This is South Los Angeles .']


# Preparing Training Data

In [0]:
def indexesFromSentence(lang, sentence):
    idxs = []
    for word in sentence.split(' '):
        try:
            idxs.append(lang.word2index[word])
        except KeyError:
            idxs.append(3)  # 3 is the id of 'UNK'
    idxs.append(EOS_token)
    return idxs


class VocabDataset(Dataset):
    def __init__(self, pairs):
        
        self.source_sent_list = [indexesFromSentence(input_lang,pair[0]) for pair in pairs]
        self.target_sent_list = [indexesFromSentence(output_lang,pair[1]) for pair in pairs]

    def __len__(self):
        return len(self.source_sent_list)
        
    def __getitem__(self, key):
        token1_idx = self.source_sent_list[key]
        token2_idx = self.target_sent_list[key]
        return [token1_idx,token2_idx, len(token1_idx), len(token2_idx)]

    
def Vocab_collate_func(batch):
    source_sent_list = []
    target_sent_list = []
    source_len_list = []
    target_len_list = []

    for datum in batch:   ### batch = sample
        source_len_list.append(datum[2])
        target_len_list.append(datum[3])
    
    max_len_src = max(source_len_list)
    max_len_trg = max(target_len_list)
    
    # padding
    for datum in batch:
        
        # source sentence processing
        padded_source = np.pad(np.array(datum[0]), 
                                pad_width=((0,max_len_src-datum[2])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        source_sent_list.append(padded_source)
        
        # target sentence processing
        padded_target = np.pad(np.array(datum[1]), 
                                pad_width=((0,max_len_trg-datum[3])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        target_sent_list.append(padded_target)
        
    #sort sentences for the batch
    sort_idx = sorted(range(len(source_len_list)), key=source_len_list.__getitem__, reverse=True)
    source_sent_list = np.array(source_sent_list)[sort_idx]
    target_sent_list = np.array(target_sent_list)[sort_idx]
    source_len_list = np.array(source_len_list)[sort_idx]
    target_len_list = np.array(target_len_list)[sort_idx]
        
    return [torch.tensor(source_sent_list).to(device), 
            torch.tensor(target_sent_list).to(device),
            torch.LongTensor(source_len_list), 
            torch.LongTensor(target_len_list)]

train_dataset = VocabDataset(pairs)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=Vocab_collate_func,
                                           shuffle=True)

val_dataset = VocabDataset(val_pairs)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                        batch_size=BATCH_SIZE,
                                        collate_fn=Vocab_collate_func,
                                        shuffle=False)


test_dataset = VocabDataset(test_pairs)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=BATCH_SIZE,
                                        collate_fn=Vocab_collate_func,
                                        shuffle=False)

# Build Encoder-Decoder

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.embedding = nn.Embedding(input_size, hidden_size)
       
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers,dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, (hidden, cell) = self.lstm(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden, output_lengths
      
    def initHidden(self,batch_size):
        return  torch.zeros(2*self.n_layers, batch_size,self.hidden_size,device=device),torch.zeros(2*self.n_layers, batch_size, self.hidden_size,device=device)
   

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,dropout_p=0.1, n_layers=1, max_length=MAX_LENGTH):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.output_size = output_size
        
        
        self.embedding = nn.Embedding(output_size, hidden_size,padding_idx=PAD_token)
        #self.gru = nn.GRU(self.hidden_size, self.hidden_size, n_layers, dropout=dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, n_layers,dropout=dropout_p)
        
        
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_seqs, hidden, batch_size):
        embedded = self.embedding(input_seqs).view(1, batch_size, -1)
        embedded = self.dropout(embedded)
        output = F.relu(embedded)
        print("#####",hidden)
        output, (hidden,cell) = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
      
    def initHidden(self):
        return torch.zeros(2, batch_size, self.hidden_size, device=device)

In [0]:
class Attn(nn.Module):
  
    """
    Some code is paraphrased from
    https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb
    """
    
    
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size

        if self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)).to(device) # B x S
        
        if self.method == 'dot':

            attn_energies = torch.matmul(encoder_outputs.permute(1,0,2), hidden.permute(1,2,0)).squeeze()
            
        if self.method == 'concat':
            hidden_expand = hidden.expand(max_len, -1, -1).permute(1, 0, 2)  # shape of (B, S, N)
            enc_cat_hid = torch.cat([encoder_outputs.permute(1,0,2), hidden_expand], dim=-1)  # shape of (B, S, 2*N)
            # After nn.Linear(2*N, N), enc_cat_hid with shape (B, S, N)
            # v is shape of (N)
            attn_energies = torch.matmul(self.attn(enc_cat_hid), self.v)  # shape of (B, S)
        # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
        # Dangerous
        if attn_energies.dim() == 1:
            attn_energies = attn_energies.unsqueeze(0)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)
    

In [0]:
class LuongAttnDecoderRNN(nn.Module):
  
  
    """
    Some code is paraphrased from
    https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb
    """
    
    
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size).to(device)
        self.embedding_dropout = nn.Dropout(dropout)
        
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers,dropout=dropout)
        
        #self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # Choose attention model
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):
        # Note: we run this one step at a time

        # Get the embedding of the current input word (last output word)
        batch_size = input_seq.size(0)
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, self.hidden_size) # S=1 x B x N

        # Get current hidden state from input word and last hidden state
        #last_hidden=[last_hidden,last_hidden]
        
        rnn_output, (hidden,cell) = self.lstm(embedded, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs;
        # apply to encoder outputs to get weighted average
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x S=1 x N

        # Attentional vector using the RNN hidden state and context vector
        # concatenated together (Luong eq. 5)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))

        # Finally predict next token (Luong eq. 6, without softmax)
        output = self.out(concat_output)

        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

# Training Model

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()

    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def train(input_tensor, target_tensor, input_lengths, target_lengths, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, clip=10.0):
    encoder_optimizer.zero_grad()  # zero out the accumulated gradient over mini-batch
    decoder_optimizer.zero_grad()
    
    
    batch_size = input_tensor.size(1)
    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)

    encoder_hidden = encoder.initHidden(batch_size)
    encoder_outputs = torch.zeros(input_lengths.max(), batch_size, encoder.hidden_size, device=device) 
 

    encoder_outputs, encoder_hidden, encoder_output_lengths = encoder(input_tensor, input_lengths, encoder_hidden)
    #encoder_outputs:  # max_len x batch_size x hidden_size
    #hidden: n_layers * 2 x batch_size x hidden_size
    loss = 0

    
    decoder_input = torch.tensor([SOS_token]*batch_size).to(device)  # decoder_input: torch.Size([1, 32])
    decoder_hidden = encoder_hidden[:decoder.n_layers] 
    # Use last (forward) hidden state from encoder
    all_decoder_outputs = Variable(torch.zeros(target_lengths.max(), batch_size, decoder.output_size)).to(device)
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_lengths.max()):
            if Attention:
                #print(decoder_hidden[0].size())
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], encoder_outputs)
     
            else:
                decoder_output, decoder_hidden = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], batch_size)
            
            decoder_input = target_tensor[di]  # Teacher forcing
            all_decoder_outputs[di] = decoder_output
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_lengths.max()):
          
            if Attention:
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], encoder_outputs)
     
            else:
                decoder_output, decoder_hidden = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], batch_size)
            
            topv, topi = decoder_output.topk(1)

            decoder_input = topi.squeeze().detach()  # detach from history as input
            all_decoder_outputs[di] = decoder_output
           
            
    # Loss calculation and backpropagation

    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_tensor.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths
    )
#     loss = loss.sum()/batch_size 
    loss.backward()
    #    ave_loss.backward()
    
    # Clip gradient norms
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()   # update parameters
    decoder_optimizer.step()

    return loss.item()

In [0]:
def trainIters(encoder, decoder, n_iters, lr_decay=True, gamma_encoder=0.9, gamma_decoder=0.9, print_every=100, plot_every=100, learning_rate_encoder=0.0005, learning_rate_decoder=0.002,evaluate_every=3000):
    start = time.time()

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate_encoder)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate_decoder)
    
    scheduler_encoder = ExponentialLR(encoder_optimizer, gamma_encoder, last_epoch=-1) 
    scheduler_decoder = ExponentialLR(decoder_optimizer, gamma_decoder, last_epoch=-1) 
    
    encoder.to(device)
    decoder.to(device)
    score_max = 0
    plot_losses = []
    validation_scores = []
    
    for epoch in range(1, n_iters + 1):
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every
        if lr_decay:
            scheduler_encoder.step()
            scheduler_decoder.step()
        
        for i, (input_sentences, target_sentences,len1,len2) in enumerate(train_loader): 
            encoder.train()
            decoder.train()
            
            input_tensor = input_sentences.transpose(0,1)   # 13*100 to 100*13
            target_tensor = target_sentences.transpose(0,1)
            loss = train(input_tensor, target_tensor, len1, len2, encoder,
                         decoder, encoder_optimizer, decoder_optimizer)
            print_loss_total += loss
            plot_loss_total += loss
            
            if i > 0 and i % evaluate_every == 0:
                bleu_score, (src_sents, sys_sents, ref_sents) = test_model(encoder, decoder, val_loader)
                print('Validation Score: {} \n source sentence {} \n predicted sentence {} \n Reference sentence: {}'.format(bleu_score,src_sents, sys_sents, ref_sents))
                validation_scores.append(bleu_score)
                
                if bleu_score > score_max:
                    score_max = bleu_score
                
                    torch.save({
                                'epoch': epoch,
                                'encoder': encoder.state_dict(),
                                'encoder_optimizer': encoder_optimizer.state_dict(),
                                'decoder': decoder.state_dict(),
                                'decoder_optimizer': decoder_optimizer.state_dict()
                                }, "/content/drive/My Drive/Colab Notebooks/saved_model/attnIs{}_hiddenSize{}_nLayer{}_batchSize{}_epoch{}_srcVocSize{}_tgtVocSize{}_lrDecay{}"\
                        .format(Attention,hidden_size,n_layers,BATCH_SIZE,n_iters,source_vocab_size,
                                target_vocab_size,lr_decay,teacher_forcing_ratio))   
                    
            if i > 0 and i % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
#                bleu_score, (sys_sents, ref_sents) = test_model(encoder, decoder, val_loader)
                print('Time: {}, Epoch: [{}/{}], Step: [{}/{}], Train Loss: {}'.format(
                    timeSince(start, i + 1/len(train_loader)), epoch, n_iters, i, 
                    len(train_loader),print_loss_avg))

            if i > 0 and i % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
                torch.save({
                            'plot_losses': plot_losses,
                            'validation_scores': validation_scores
                            }, "/content/drive/My Drive/Colab Notebooks/saved_scores/attnIs{}_hiddenSize{}_nLayer{}_batchSize{}_epoch{}_srcVocSize{}_tgtVocSize{}_lrDecay{}"\
                    .format(Attention,hidden_size,n_layers,BATCH_SIZE,n_iters,source_vocab_size,
                            target_vocab_size,lr_decay,teacher_forcing_ratio))   
                
            torch.cuda.empty_cache()    
        print("plot_losses:",plot_losses)
        print("validation_scores:",validation_scores)
    showPlot(plot_losses)
    showPlot(validation_scores)

# Plotting results

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation

In [0]:
class beam_search():
  
    """
    Some code is paraphrased from
    https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/translate/beam.py
    """
    
    def __init__(self, encoder, decoder, max_length, beam_size, attention = False, dynamic_sentence_length = False):
        """
        Args:
            encoder: the encoder network
            decoder: the decoder network
            attention: boolean. True if using attention
            max_length: int. max sentence length produced
            beam_size: int.
        """    
        super(beam_search, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.attention = attention
        self.max_length = max_length
        self.beam_size = beam_size
        self.dynamic_sentence_length = dynamic_sentence_length
        
        
    def search(self, encoder_outputs, decoder_input, decoder_hidden, source_sentence_length = 0):
        """
        Args:
            encoder_output: output of encoder, used for attention. shape: 1 x 1 x hidden_size
            decoder_input: SOS token (e.g. torch.tensor([[SOS_token]], device=device))
            decoder_hidden: last encoder hidden vector. 
            decoder_cell_state: last encoder cell state.
        """
        decoder_input_cand = {}
        decoder_output_cand = {}
        decoder_hidden_cand = {}
        decoded_words_cand = {k:[] for k in range(self.beam_size)}
        decoded_sentences_prob = {k:0 for k in range(self.beam_size)} # create decoded_sentences_prob
        final_sent = []
        final_score = []
        
        ## INIT
        if self.attention == True:
            #decoder_hidden = [decoder_hidden,decoder_hidden]
            
            
            
            
            
            decoder_output, decoder_hidden, decoder_attention = self.decoder(decoder_input.contiguous(), [decoder_hidden.contiguous(),decoder_hidden.contiguous()], encoder_outputs)
        
            decoder_output = F.log_softmax(decoder_output, dim=1)
            topv, topi = decoder_output.data.topk(self.beam_size)
        else: 
            decoder_output, decoder_hidden = self.decoder(decoder_input.contiguous(),decoder_hidden.contiguous())
            decoder_output = F.log_softmax(decoder_output, dim=2)
            topv, topi = decoder_output.data.topk(self.beam_size)
            topv, topi = topv.squeeze(0), topi.squeeze(0)
            
        #print(decoder_input, decoder_output.shape, decoder_output.data.shape)
        for i in range(self.beam_size):
            decoded_words_cand[i].append(topi.squeeze()[i].item())
            decoder_input_cand[i] = topi.squeeze()[i].detach()
            decoder_hidden_cand[i] = decoder_hidden
            #print(decoder_input_cand[i].view(1,-1))
            decoded_sentences_prob[i] += topv.squeeze()[i].detach() # calculate log probability (multiplication becomes addition)
            
        ## BEAM-SEARCH
        word_cnt = 0
        max_length = 2*source_sentence_length if self.dynamic_sentence_length else self.max_length
        while (bool(decoder_hidden_cand)) & (word_cnt <= max_length):
            word_cnt += 1
            topi = {}
            avail_keys = list(decoder_hidden_cand.keys())
            score_all = []
            for b in avail_keys:
                if self.attention == True:
                    
                    decoder_output, decoder_hidden_cand[b],decoder_attn  = self.decoder(decoder_input_cand[b].unsqueeze(0), [decoder_hidden_cand[b],decoder_hidden_cand[b]],encoder_outputs)
                    decoder_output_cand[b] = F.log_softmax(decoder_output, dim=1)
                    topv, topi[b] = decoder_output_cand[b].data.topk(len(decoder_hidden_cand))
                else:
                    decoder_output, decoder_hidden_cand[b] = self.decoder(decoder_input_cand[b].view(1,-1),decoder_hidden_cand[b])
                    decoder_output_cand[b] = F.log_softmax(decoder_output, dim=2)
                    topv, topi_b = decoder_output_cand[b].data.topk(len(decoder_hidden_cand))
                    topv, topi[b] = topv.squeeze(0), topi_b.squeeze(0)
                
                #print(decoder_output, topv)
                #print(topv, topi[b])
                score_all.extend((topv+decoded_sentences_prob[b]).tolist()[0])
                
            score_all = np.array(score_all)   
            max_cand = score_all.argsort()[-len(decoder_hidden_cand):][::-1]
            decoded_sent_score = score_all[max_cand]

            cand_sentences = {}
            cand_hiddens = {}
            cand_cell_states = {}
            keys_to_rm = []
            
            for j in range(len(max_cand)):
                prev_cand_id = avail_keys[int(np.floor(max_cand[j]/len(decoder_hidden_cand)))]
                if topi[prev_cand_id].squeeze().dim() == 0:
                    next_id = topi[prev_cand_id].squeeze()
                else:
                    next_id = topi[prev_cand_id].squeeze()[max_cand[j] % len(decoder_hidden_cand)]
                s_cand = decoded_words_cand[prev_cand_id].copy()
                s_cand.append(next_id.item())
                cand_sentences[j] = s_cand
                
                h_cand = decoder_hidden_cand[prev_cand_id]
                cand_hiddens[j] = h_cand
                
                decoder_input_cand[j] = next_id.detach()   

                decoded_sentences_prob[j] = decoded_sent_score[j] # update decoded_sentences_prob

                
            decoded_words_cand = cand_sentences
            decoder_hidden_cand = cand_hiddens
            
            #print(decoded_sentences_prob)
            for key, s in decoded_words_cand.items():
                if EOS_token in s:
                    final_sent.append(s)
                    #final_score.append(decoded_sent_score[key])
                    final_score.append(decoded_sentences_prob[key]) # use the joint probability. actually, same as using decoded_sent_score..
                    keys_to_rm.append(key)
                    
            for k in keys_to_rm:
                decoder_hidden_cand.pop(k)
                decoded_words_cand.pop(k)

        if len(final_score) == 0:
            max_prob = decoded_sentences_prob[0]
            max_prob_id = 0
            for k in decoded_sentences_prob.keys():
                if decoded_sentences_prob[k] > max_prob: 
                    max_prob_id = k
                    max_prob = decoded_sentences_prob[k]
            final_score.append(max_prob)
            final_sent.append(decoded_words_cand[max_prob_id])
                
        return final_sent, final_score

In [0]:
def get_batch_outputs(encoder, decoder, input_sentences, input_lengths, output_lengths): 
    with torch.no_grad():
        input_tensor = input_sentences.transpose(0,1).to(device)   # 32*100 to 100*32
        batch_size = input_tensor.size(1)
        encoder_hidden = encoder.initHidden(batch_size)
        encoder_outputs, encoder_hidden,encoder_output_lengths = encoder(input_tensor, input_lengths, encoder_hidden)
       
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        

        decoder_input = Variable(torch.tensor([SOS_token]*batch_size)).to(device)  # decoder_input: torch.Size([1, 32])
        decoded_words = np.empty((output_lengths.max(), batch_size), dtype=object)

        for di in range(output_lengths.max()):
            if Attention:
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], encoder_outputs)

            else:
                decoder_output, decoder_hidden = decoder(
                    decoder_input, [decoder_hidden,decoder_hidden], batch_size)

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach().to(device)  # detach from history as input

            decoded_words[di:] = np.array(['<EOS>' if idx==EOS_token else output_lang.index2word[idx] for idx in decoder_input.tolist()])

        return decoded_words.transpose()

In [0]:
def pad(l, max_length):
    while len(l) < max_length + 2:
        l.append(PAD_token)
    return l

def get_beam_batch_outputs(encoder, decoder, input_sentences, input_lengths): #####
    with torch.no_grad():
        input_tensor = input_sentences.transpose(0,1).to(device)   # 32*100 to 100*32
        batch_size = input_tensor.size(1)
        encoder_hidden = encoder.initHidden(batch_size)
        encoder_outputs, encoder_hidden,encoder_output_lengths = encoder(input_tensor, input_lengths, encoder_hidden)
       
        decoder_hidden = encoder_hidden[:decoder.n_layers]   
        
        my_beam_search = beam_search(encoder, decoder,input_sentences.max().item(), beam_size, True, dynamic_sentence_length = dynamic_sentence_length)
        beam_search_result = []
        for i in range(batch_size):
            decoder_input = torch.tensor([SOS_token], device=device, requires_grad=False).unsqueeze(0)#.view(1,-1) # take care of different input shape
            sentences, probs = my_beam_search.search(encoder_outputs[:,i,:].unsqueeze(1), decoder_input, 
                                                     decoder_hidden[:,i,:].unsqueeze(1), None)
            
            beam_search_result.append(sentences[probs.index(max(probs))])

        padded_beam_search_result = []

        max_length = 0
        for each in beam_search_result:
            if len(each) > max_length:
                max_length = len(each)

        for each in beam_search_result:
            padded_beam_search_result.append(pad(each, max_length))

        batch_sentences = []
        
        for sentence in padded_beam_search_result:
            sentence = [output_lang.index2word[k] for k in sentence]
            
            try:
                end_idx = sentence.index('<EOS>')
                batch_sentences.append(' '.join(sentence[:end_idx]))
            except ValueError:
                batch_sentences.append(' '.join(sentence))

    return batch_sentences

In [0]:
def test_model(encoder, decoder, loader, search_method = 'greedy'):
    
    encoder.eval()
    decoder.eval()
    
    score = []
    src_sentences = []
    sys_sentences = []
    ref_sentences = []
    encoder.train(False)
    decoder.train(False)
    for i, (input_sentences, target_sentences, len1, len2) in enumerate(loader):
        for sentence in target_sentences:
            trg_list = []
            for idx in sentence:
                if idx.item() == EOS_token:
                    break
                else:
                    trg_list.append(output_lang.index2word[idx.item()])
            ref_sentences.append(' '.join(trg_list))
        for sentence in input_sentences:
            src_list = []
            for idx in sentence:
                if idx.item() == EOS_token:
                    break
                else:
                    src_list.append(input_lang.index2word[idx.item()])
            src_sentences.append(' '.join(src_list))

        #ref_sentences.append(' '.join(sent) for sent in target_sentences)
        #src_sentences.append(' '.join(sent) for sent in input_sentences)
        batch_size = input_sentences.size(0)
        if search_method == 'greedy':
            for sentence in get_batch_outputs(encoder, decoder, input_sentences, len1, len2):
                try:
                    end_idx = sentence.tolist().index('<EOS>')
                    sys_sentences.append(' '.join(sentence[:end_idx]))
                except ValueError:
                    sys_sentences.append(' '.join(sentence))
                    
        elif search_method == 'beam':
            translation_output = get_beam_batch_outputs(encoder, decoder, input_sentences, len1)
            sys_sentences.extend(translation_output)
            
    encoder.train(True)
    decoder.train(True) 
    
    score = corpus_bleu(sys_sentences,[ref_sentences], smooth="floor", smooth_floor=0.01, lowercase=False, use_effective_order=True, tokenize=DEFAULT_TOKENIZER).score
    return score, (src_sentences[0:5], sys_sentences[0:5], ref_sentences[0:5])


# TRAINING AND EVALUATING

In [0]:
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, n_layers=n_layers).to(device)
if Attention:
    decoder1 = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.n_words, dropout=dropout_p, n_layers=n_layers).to(device)
else:
    decoder1 = DecoderRNN(hidden_size, output_lang.n_words, dropout_p=0)

trainIters(encoder1, decoder1, n_iters=n_epochs, print_every=print_every, plot_every=plot_every, evaluate_every=evaluate_every, learning_rate_encoder=lr_rate_en, learning_rate_decoder=lr_rate_de, lr_decay=lr_decay, gamma_encoder=gamma_decoder,
          gamma_decoder=gamma_decoder)

In [0]:
%matplotlib inline
showPlot(validation_scores)

# Resume Model and Test

In [0]:
encoder2 = EncoderRNN(input_lang.n_words, hidden_size,n_layers=n_layers).to(device)
if not Attention:
    decoder2 = DecoderRNN(hidden_size, output_lang.n_words, dropout_p = 0).to(device)
else:
    decoder2 = LuongAttnDecoderRNN(attn_model, hidden_size, output_lang.n_words, dropout=dropout_p, n_layers=n_layers).to(device)
    
    
# # encoder_optimizer = optim.Adam(encoder1.parameters(), lr=0.003)
# # decoder_optimizer = optim.Adam(attn_decoder1.parameters(), lr=0.003)

checkpoint = torch.load('/content/drive/My Drive/Colab Notebooks/saved_model/attnIsTrue_hiddenSize512_nLayer4_batchSize64_epoch20_srcVocSize4300_tgtVocSize39000_lrDecayTrue')
encoder2.load_state_dict(checkpoint['encoder'])
decoder2.load_state_dict(checkpoint['decoder'])
# # encoder_optimizer.load_state_dict(checkpoint['encoder_optimizer'])
# # decoder_optimizer.load_state_dict(checkpoint['decoder_optimizer'])
# # epoch = checkpoint['epoch']

encoder2.eval()
decoder2.eval()
# # encoder1.train()
# # attn_decoder1.train()

LuongAttnDecoderRNN(
  (embedding): Embedding(39004, 512)
  (embedding_dropout): Dropout(p=0.1)
  (lstm): LSTM(512, 512, num_layers=4, dropout=0.1)
  (concat): Linear(in_features=1024, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=39004, bias=True)
  (attn): Attn()
)

# Resume Loss and Bleu

In [0]:
# score = torch.load('saved_scores/attnIsTrue_hiddenSize1024_nLayer2_batchSize64_epoch20_srcVocSize22000_tgtVocSize39000_lrDecayFalse_teacherF1')
# print(score['plot_losses'])
# print(score['validation_scores'])

# greedy

In [0]:
search_method = 'greedy'
print(test_model(encoder2, decoder2, val_loader))
print(test_model(encoder2, decoder2, test_loader))

(12.291619742332031, (['有 着 如 此 肥 沃 的 山 谷 ， 当 地 人 居 然 不 耕 种 ， 我 们 对 此 感 到 十 分 惊 奇 。', '我 们 这 些 西 方 援 助 国 在 过 去 5 0 年 里 向 非 洲 大 陆 投 入 了 两 万 亿 美 元 。', '我 们 都 知 道 自 己 正 冒 着 生 命 的 危 险 - 老 师 ， 学 生 以 及 家 长 都 很 明 白', '让 我 兴 奋 的 是 ， 我 看 见 我 学 校 中 的 学 生 能 够 把 握 机 遇 ， 志 向 远 大 。', '但 我 们 并 没 有 问 他 们 原 因 ， 而 仅 仅 是 感 叹 道 ： " 幸 亏 我 们 来 了 。 "'], ["And there 's a very rare valley and we were very surprising at all and we 're so surprising about it .", "We 've got a national farm in the last 50 years to reach Africa to be over the last 50 million dollars .", 'We all know that the risk of life teachers and fathers and family were very aware of them .', "I 'm excited to see that I see my school students to drive the arm .", "But we didn 't ask them because it 's just the God We come . "], ['And we were amazed that the local people in such a fertile valley would not have any agriculture .', 'We Western donor countries have given the African continent two trillion American dollars in the last 50 years .', 

(10.844923523139148, (['想 象 着 他 作 为 一 名 共 产 主 义 者 进 入 胡 志 明 市 面 对 着 完 全 被 抛 弃 的 一 生', '我 母 亲 ， M a i ， 1 8 岁 丧 父 经 历 了 一 系 列 婚 姻 ， 有 了 两 个 小 女 儿 。', '所 以 今 天 我 们 要 讨 论 一 下 为 什 么 一 些 视 频 会 风 靡 甚 至 会 变 得 重 要 。', '这 只 " N y a n C a t " 是 一 种 连 环 动 画 片 配 合 连 续 的 背 景 音 乐 。', '所 以 现 在 有 上 万 个 恶 搞 版 " 周 五 " 在 Y o u T u b e 上 传 播 。'], ['Imagine he became a famous nut to get the accredited and he was absolutely abandoned .', 'My mother Dr 18 18 years old and I had two of them who had two four daughters .', "So today we 're going to talk about why some of these videos are going to be important .", 'This is a spearing of a form of a form of animation and shaped through the background of a warehouse .', 'So there are tens of the number of missions on YouTube .'], ['Imagine him as the communists enter <UNK> confronting the fact that his life had been a complete waste .', 'My mother <UNK> was 18 when her father died already in an arranged marriage already with two small girls .', "So we 're going to

# Beam

In [0]:
search_method = 'beam'
for beam_size in range(2,15):
    print("beam_size: ",beam_size)
    print(test_model(encoder2, decoder2, val_loader,search_method))
    print(test_model(encoder2, decoder2, test_loader,search_method))